In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

2023-09-03 12:11:14.054388: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 12:11:14.168132: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 12:11:14.169007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 12:11:14.681595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Matplotlib is building the font cache; this may take a moment.


In [2]:
data = pd.read_csv("kc_house_data.csv")

In [3]:
data.shape

(21613, 21)

In [4]:
pd.options.display.max_columns = 25

In [5]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [6]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [7]:
# STEP03 資料前處理 : pdf.72
data['year'] = pd.to_numeric(data['date'].str.slice(0, 4))
data['month'] = pd.to_numeric(data['date'].str.slice(4, 6))
data['day'] = pd.to_numeric(data['date'].str.slice(6, 8))

data.drop(['id'], axis ="columns", inplace = True)
data.drop(['date'], axis ="columns", inplace = True)

data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,13
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,9
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,25
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,9
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,18


In [8]:
#=== 分割資料 ===
# shape[0] -> 返回行數; shape[1] -> 返回列數
data_num = data.shape[0] 

# 取得一筆與data數量相同的亂數索引(隨機排序)，主要目的是用於打散資料
indexes = np.random.permutation(data_num)

# 亂數索引值分為Train, Valiadation & Test，比例 = 6:2:2
train_indexes = indexes[:int(data_num * 0.6)]
val_indexes = indexes[int(data_num * 0.6) : int(data_num * 0.8)]
test_indexes = indexes[int(data_num * 0.8):]

# 透過索引值從data取出訓練資料、驗證資料和測試資料
train_data = data.loc[train_indexes]
val_data = data.loc[val_indexes]
test_data = data.loc[test_indexes]

In [9]:
# STEP04 標準化 pdf.73
"""
將不同的資料縮放到相同大小。
通常會將輸入資料縮放到0 ~ 1 or -1 ~ 1之間，這個過程稱之「資料標準化」(Data Normalization)
本範例使用「標準分數」(Standard Score, z-score)來將資料標準化
經過z-score標準化後的資料，都會聚集在0附近且標準差為1
"""
train_validation_data = pd.concat([train_data, val_data])
mean = train_validation_data.mean()
std = train_validation_data.std()
train_data = (train_data - mean) / std
val_data = (val_data - mean) / std


In [10]:
# STEP05 建立Numpy array 格式的訓練資料 pdf.73
# drop's attr. "axis" value incl. : 0, 1, 'index', 'columns'
x_train = np.array(train_data.drop('price', axis = 'columns'))
y_train = np.array(train_data['price'])
x_val = np.array(val_data.drop('price', axis = 'columns'))
y_val = np.array(val_data['price'])

x_train.shape

(12967, 21)

In [11]:
# STEP06 建立並訓練網路模型 : use ReLU as hidden layer's activation function
model = keras.Sequential(name='model-1')
model.add(layers.Dense(64, activation='relu', input_shape=(21,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
model.summary()



Model: "model-1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1408      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5633 (22.00 KB)
Trainable params: 5633 (22.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# 設定訓練使用的優化器、損失函數和指標函數
model.compile(keras.optimizers.Adam(0.001),
             loss = keras.losses.MeanSquaredError(),
             metrics = [keras.metrics.MeanAbsoluteError()])
# 建立模型儲存目錄
model_dir = 'lab2-logs/models/'
os.makedirs(model_dir)
log_dir = os.path.join('lab2-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + 'Best-model-1.h5',
                                            monitor = 'val_mean_absolute_error',
                                            save_best_only = True,
                                            mode = 'min')
# 訓練網路模型
history = model.fit(x_train, y_train,
                   batch_size = 64,
                   epochs = 300,
                   validation_data = (x_val, y_val),
                   callbacks = [model_cbk, model_mckp])

FileExistsError: [Errno 17] File exists: 'lab2-logs/models/'

In [ ]:
# 歷史紀錄
history.history.keys()

In [ ]:
# 繪製損失值的折線圖
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'validation')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc = 'upper right')

In [ ]:
# 繪製效能指標的折線圖
plt.plot(history.history['mean_absolute_error'], label = 'train')
plt.plot(history.history['val_mean_absolute_error'], label = 'validation')
plt.ylabel('metrics')
plt.xlabel('epochs')
plt.legend(loc = 'upper right')

In [ ]:
# 測試資料的誤差百分比
model.load_weights('lab2-logs/models/Best-model-1.h5')
y_test = np.array(test_data['price'])
test_data = (test_data - mean) / std
x_test = np.array(test_data.drop('price', axis = 'columns'))
y_pred =model.predict(x_test)
y_pred = np.reshape(y_pred * std['price'] + mean['price'], y_test.shape)
percentage_error = np.mean(np.abs(y_test - y_pred)) / np.mean(y_test) * 100
print("Model_1 Percentage Error: {:.2f}%".format(percentage_error))

In [13]:
# For test 
"""
x_train = np.array(train_data.drop('price', axis = 'columns'))
y_train = np.array(train_data['price'])
x_val = np.array(val_data.drop('price', axis = 'columns'))
y_val = np.array(val_data['price'])
"""
print(train_data)
print("===")
print(y_train)

          price  bedrooms  bathrooms  sqft_living  sqft_lot    floors  \
5665  -0.067442 -0.393932  -0.470664    -0.619560 -0.169828 -0.913393   
5074  -0.288888  0.670300   0.499467     0.052725 -0.115363  0.935637   
11258 -0.439285 -0.393932  -0.147287     1.115368  0.771257  0.935637   
9886  -0.818416 -0.393932  -0.794041    -0.456910 -0.176236 -0.913393   
3508   0.420788  0.670300   0.499467     1.006935 -0.089271  0.935637   
...         ...       ...        ...          ...       ...       ...   
13474 -0.152412  0.670300  -0.470664    -0.706306 -0.276298  0.935637   
9636   0.091881  0.670300  -0.794041    -0.706306 -0.217758  0.011122   
11703 -0.673751 -0.393932   1.146220    -0.446067 -0.275683  0.011122   
12892 -0.905761 -1.458165  -1.440794    -1.096665 -0.224422 -0.913393   
19413 -0.889384 -1.458165  -1.440794    -0.912329 -0.196740 -0.913393   

       waterfront      view  condition     grade  sqft_above  sqft_basement  \
5665    -0.087708 -0.304434   0.906421  0.28